#### This notebook will download MODIS data in order to create predicted time-series Raster from a trained model 

In [ ]:
import ee
import time
import random
from google.oauth2.credentials import Credentials
from multiprocessing import Pool

In [ ]:
# Details to get Google Earth Engine API access
# https://www.kaggle.com/code/pcjimmmy/how-to-get-the-secret-and-tokens-and-access
credentials = Credentials(
        None,
        refresh_token = "MY_REFRESH_TOKEN",
        token_uri=ee.oauth.TOKEN_URI,
        client_id="MY_CLIENT_ID",
        client_secret="MY_CLIENT_SECRET",
        scopes=ee.oauth.SCOPES)

In [ ]:
ee.Initialize(credentials=credentials)

#### Define the `year_` variable you want to download MODIS data for
#### On Google Earth Engine upload shapefile from /data/NC_CASC_region
#### Provide the GEE path of this shapefile to `states` variable

In [ ]:
year_ = 2003
year_to_calculate_start = year_
year_to_calculate_end = year_

states = ee.FeatureCollection("path/NC_CASC_region")
modis_product = ee.ImageCollection("MODIS/061/MCD43A4")
aqua_primary_gpp_npp = ee.ImageCollection("MODIS/061/MYD17A3HGF")
LAI_FPAR = ee.ImageCollection("MODIS/061/MCD15A3H")
terra_primary_gpp_npp = ee.ImageCollection("MODIS/061/MOD17A3HGF")
ecoregionsL3 = ee.FeatureCollection("EPA/Ecoregions/2013/L3")
nlcd2001 = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD").filter(ee.Filter.eq('system:index', '2001')).first().select("landcover")

lai_product = LAI_FPAR.select("Lai", "FparLai_QC")
fpar_product = LAI_FPAR.select("Fpar", "FparLai_QC")
fpar_lai_projection = lai_product.first().projection()
terra_primary_npp = terra_primary_gpp_npp.select("Npp")
aqua_primary_npp = aqua_primary_gpp_npp.select("Npp")
terra_primary_gpp = terra_primary_gpp_npp.select("Gpp")
aqua_primary_gpp = aqua_primary_gpp_npp.select("Gpp")

#### Since we did not use GCP buckets to save the data, and used Google Drive instead, create a folder with the value of `GDRIVE_FOLDER` variable. For E.g. we created a folder called **MODIS_DATA_FOR_SOUTH_DAKOTA_2003**. The downloaded MODIS data was stored in this folder. Make sure you do all these Google operations on the same account. By default GEE has access to your drive

#### We did this for 7 NC states, and for years 2003 to 2017. Feel free to make copies of these notebooks and run parallely for different years if you want to speed up the process

#### We download data if the pixel is Deciduous forest, Evergreen forest or Mixed forest. This info is mapped from NLCD data (2001)

In [ ]:
GDRIVE_FOLDER = "MODIS_DATA_FOR_SOUTH_DAKOTA_2003"

In [ ]:
def make_regions_to_extract_modis(state_id):
    def createSmallRegion(x, y):
      x1 = ee.Number(x)
      y1 = ee.Number(y)
      x2 = x1.add(dx)
      y2 = y1.add(dy)
      return ee.Geometry.Rectangle([x1, y1, x2, y2])

    def upsampleNlcdInRegion(region):
      geometry = region.geometry()
      upsampled = clippedNlcd2001.reduceResolution(reducer=ee.Reducer.mode(), bestEffort=True, maxPixels=1024).reproject(crs=modis_projection, scale=modis_projection.nominalScale()).clip(geometry)
      return upsampled.set('region', region.get('index'))
    
    co = ee.FeatureCollection(states.toList(states.size()).slice(state_id, state_id+1))

    # Get the projection of the MODIS product
    modis_projection = modis_product.first().projection()

    # Clip NLCD2001 to the region of interest
    clippedNlcd2001 = nlcd2001.clipToCollection(co)

    # Define parameters for splitting the state into small blocks
    dx = 0.15
    dy = 0.15

    # Get the bounds of the region of interest
    bounds = ee.List(co.geometry().bounds(ee.ErrorMargin(0.001)).coordinates().get(0))
    xmin = ee.Number(ee.List(bounds.get(0)).get(0))
    ymin = ee.Number(ee.List(bounds.get(0)).get(1))
    xmax = ee.Number(ee.List(bounds.get(2)).get(0))
    ymax = ee.Number(ee.List(bounds.get(2)).get(1))

    # Create a feature collection of small regions
    xsteps = xmax.subtract(xmin).divide(dx).ceil()
    ysteps = ymax.subtract(ymin).divide(dy).ceil()
    smallRegions = ee.FeatureCollection(
      ee.List.sequence(0, xsteps.subtract(1)).map(lambda i:
        ee.List.sequence(0, ysteps.subtract(1)).map(lambda j:
          ee.Feature(createSmallRegion(xmin.add(ee.Number(i).multiply(dx)), ymin.add(ee.Number(j).multiply(dy))).intersection(co.geometry(), ee.ErrorMargin(0.001))).set('index', ee.String(i).cat('_').cat(ee.String(j)))
        )
      ).flatten()
    )

    smallRegions = smallRegions.filter(ee.Filter.And(ee.Filter.notNull(['system:index']), ee.Filter.neq('index', '')))
    smallRegions = smallRegions.filter(ee.Filter.intersects('.geo', co.geometry()));

    # Upsample NLCD within each small block
    upsampledRegions = smallRegions.map(upsampleNlcdInRegion)

    # Mask out non-forest landcover
    upsampledRegions = upsampledRegions.map(lambda image:ee.Image(image).select("landcover").updateMask(ee.Image(image).select("landcover").eq(41).Or(ee.Image(image).select("landcover").eq(42)).Or(ee.Image(image).select("landcover").eq(43))))
    
    return upsampledRegions

In [ ]:
def run_mp(data):
  state_id, index_to_run, upsampledRegions = data[0], data[1], data[2]
  try:
    def calculateNDVI(image):
      ndvi = image.normalizedDifference(["Nadir_Reflectance_Band2", "Nadir_Reflectance_Band1"])
      return image.addBands(ndvi.rename("NDVI"))

    def calculateEVI(image):
      G = 2.5
      C1 = 6
      C2 = 7.5
      L = 1
      red = image.select('Nadir_Reflectance_Band1')
      nir = image.select('Nadir_Reflectance_Band2')
      blue = image.select('Nadir_Reflectance_Band3')
      evi = nir.multiply(0.0001).subtract(red.multiply(0.0001)).multiply(G).divide(nir.multiply(0.0001).add(red.multiply(0.0001).multiply(C1)).subtract(blue.multiply(0.0001).multiply(C2)).add(L))
      return evi.rename('EVI')

    def calculateNDWI(image):
      ndwi = image.normalizedDifference(["Nadir_Reflectance_Band2", "Nadir_Reflectance_Band5"])
      return ndwi.rename("NDWI")

    def clipFunction(image):
      return image.clip(roi)

    def filterBandsNDVI(image):
      mask_red = image.select("BRDF_Albedo_Band_Mandatory_Quality_Band1").eq(0)
      mask_nir = image.select("BRDF_Albedo_Band_Mandatory_Quality_Band2").eq(0)
      return image.updateMask(mask_red).updateMask(mask_nir)

    def filterBandsEVI(image):
      mask_red = image.select("BRDF_Albedo_Band_Mandatory_Quality_Band1").eq(0)
      mask_nir = image.select("BRDF_Albedo_Band_Mandatory_Quality_Band2").eq(0)
      mask_blue = image.select("BRDF_Albedo_Band_Mandatory_Quality_Band3").eq(0)
      return image.updateMask(mask_nir).updateMask(mask_red).updateMask(mask_blue)

    def filterBandsNDWI(image):
      mask_nir = image.select("BRDF_Albedo_Band_Mandatory_Quality_Band2").eq(0)
      mask_swir = image.select("BRDF_Albedo_Band_Mandatory_Quality_Band5").eq(0)
      return image.updateMask(mask_nir).updateMask(mask_swir)

    def maskCloudsInFPAR(image):
      qc = image.select('FparLai_QC')
      cloudState = qc.bitwiseAnd(int('11000', 2)).rightShift(3)
      mask = cloudState.eq(0)
      return image.select("Fpar").updateMask(mask)

    def maskCloudsInLAI(image):
      qc = image.select('FparLai_QC')
      cloudState = qc.bitwiseAnd(int('11000', 2)).rightShift(3)
      mask = cloudState.eq(0)
      return image.select("Lai").updateMask(mask)

    def samplePointsFromFeatureWithAll(feature):
      return feature.set({
          'eco': ecoregionsL3.filterBounds(feature.geometry()).first().get('l3_key'),
          'NDVI_mean': feature.get('NDVI_mean'),
          'EVI_mean': feature.get('EVI_mean'),
          'NDWI_mean': feature.get('NDWI_mean'),
          'Fpar_mean': feature.get('Fpar_mean'),
          'Lai_mean': feature.get('Lai_mean'),
          'elevation': feature.get('elevation'),
          'aspect': feature.get('aspect'),
          'slope': feature.get('slope'),
          'NDVI_mean_annual': feature.get('NDVI_mean_annual'),
          'EVI_mean_annual': feature.get('EVI_mean_annual'),
          'NDWI_mean_annual': feature.get('NDWI_mean_annual'),
          'Fpar_mean_annual': feature.get('Fpar_mean_annual'),
          'Lai_mean_annual': feature.get('Lai_mean_annual'),
          'Terra_gpp_mean_annual': feature.get('Terra_gpp_mean_annual'),
          'Aqua_gpp_mean_annual': feature.get('Aqua_gpp_mean_annual'),
          'Terra_npp_mean_annual': feature.get('Terra_npp_mean_annual'),
          'Aqua_npp_mean_annual': feature.get('Aqua_npp_mean_annual'),
        })

    # 1. Data preparation and clipping

    # Define the region of interest
    co = ee.FeatureCollection(states.toList(states.size()).slice(state_id, state_id+1))

    # Get the projection of the MODIS product
    modis_projection = modis_product.first().projection()
    # Select the forest region
    forest_in_co = ee.Image(upsampledRegions.toList(upsampledRegions.size()).get(index_to_run))

    # Get the region of interest (ROI)
    roi = forest_in_co.geometry()

    # 2. MODIS data processing

    # Define start and end dates
    start_date = ee.Date.fromYMD(year_to_calculate_start, 1, 1)
    end_date = ee.Date.fromYMD(year_to_calculate_end, 12, 31)

    # Calculate NDVI, EVI, and NDWI for MODIS
    modis_ndvi_mean =  modis_product.map(clipFunction).filterDate(start_date, end_date).filter(ee.Filter.calendarRange(6, 9, 'month')).map(filterBandsNDVI).map(calculateNDVI).select("NDVI").reduce(ee.Reducer.mean()).rename("NDVI_mean").reproject(crs=modis_projection)
    modis_evi_mean =   modis_product.map(clipFunction).filterDate(start_date, end_date).filter(ee.Filter.calendarRange(6, 9, 'month')).map(filterBandsEVI).map(calculateEVI).select("EVI").reduce(ee.Reducer.mean()).rename("EVI_mean").reproject(crs=modis_projection)
    modis_ndwi_mean =  modis_product.map(clipFunction).filterDate(start_date, end_date).filter(ee.Filter.calendarRange(6, 9, 'month')).map(filterBandsNDWI).map(calculateNDWI).select("NDWI").reduce(ee.Reducer.mean()).rename("NDWI_mean").reproject(crs=modis_projection)

    # Calculate FPAR and LAI for MODIS
    modis_fpar_mean =  fpar_product.map(clipFunction).filterDate(start_date, end_date).filter(ee.Filter.calendarRange(6, 9, 'month')).map(maskCloudsInFPAR).select("Fpar").reduce(ee.Reducer.mean()).rename("Fpar_mean").reproject(crs=modis_projection)
    modis_lai_mean =  lai_product.map(clipFunction).filterDate(start_date, end_date).filter(ee.Filter.calendarRange(6, 9, 'month')).map(maskCloudsInLAI).select("Lai").reduce(ee.Reducer.mean()).rename("Lai_mean").reproject(crs=modis_projection)

    # Calculate annual means of NDVI, EVI, NDWI, FPAR, and LAI
    modis_ndvi_mean_annual =  modis_product.map(clipFunction).filterDate(start_date, end_date).map(filterBandsNDVI).map(calculateNDVI).select("NDVI").reduce(ee.Reducer.mean()).rename("NDVI_mean_annual").reproject(crs=modis_projection)
    modis_evi_mean_annual = modis_product.map(clipFunction).filterDate(start_date, end_date).map(filterBandsEVI).map(calculateEVI).select("EVI").reduce(ee.Reducer.mean()).rename("EVI_mean_annual").reproject(crs=modis_projection)
    modis_ndwi_mean_annual =  modis_product.map(clipFunction).filterDate(start_date, end_date).map(filterBandsNDWI).map(calculateNDWI).select("NDWI").reduce(ee.Reducer.mean()).rename("NDWI_mean_annual").reproject(crs=modis_projection)
    modis_fpar_mean_annual =  fpar_product.map(clipFunction).filterDate(start_date, end_date).map(maskCloudsInFPAR).select("Fpar").reduce(ee.Reducer.mean()).rename("Fpar_mean_annual").reproject(crs=modis_projection)
    modis_lai_mean_annual =  lai_product.map(clipFunction).filterDate(start_date, end_date).map(maskCloudsInLAI).select("Lai").reduce(ee.Reducer.mean()).rename("Lai_mean_annual").reproject(crs=modis_projection)

    # Calculate annual means of GPP and NPP for TERRA and AQUA
    terra_gpp_mean_annual = terra_primary_gpp.map(clipFunction).filterDate(start_date, end_date).reduce(ee.Reducer.mean()).multiply(0.0001).rename("Terra_gpp_mean_annual").reproject(crs=modis_projection)
    aqua_gpp_mean_annual =  aqua_primary_gpp.map(clipFunction).filterDate(start_date, end_date).reduce(ee.Reducer.mean()).multiply(0.0001).rename("Aqua_gpp_mean_annual").reproject(crs=modis_projection)
    terra_npp_mean_annual = terra_primary_npp.map(clipFunction).filterDate(start_date, end_date).reduce(ee.Reducer.mean()).multiply(0.0001).rename("Terra_npp_mean_annual").reproject(crs=modis_projection)
    aqua_npp_mean_annual =  aqua_primary_npp.map(clipFunction).filterDate(start_date, end_date).reduce(ee.Reducer.mean()).multiply(0.0001).rename("Aqua_npp_mean_annual").reproject(crs=modis_projection)

    # 3. DEM and terrain data processing

    # Load and process DEM
    dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
    dem = dem.updateMask(dem.gt(0)).clip(roi)
    slope = ee.Terrain.slope(dem)
    aspect = ee.Terrain.aspect(dem)

    # Resample DEM, slope, and aspect
    dem_resampled = dem.reduceResolution(reducer=ee.Reducer.mean(), maxPixels=1024 * 10).reproject(crs=modis_projection)
    slope_resampled = slope.reduceResolution(reducer=ee.Reducer.mean(), maxPixels=1024 * 10).reproject(crs=modis_projection)
    aspect_resampled = aspect.reduceResolution(reducer=ee.Reducer.mean(), maxPixels=1024 * 10).reproject(crs=modis_projection)

    # 4. Combine all data into a single image

    combined_image = modis_ndvi_mean.addBands(modis_evi_mean).addBands(modis_ndwi_mean).addBands(modis_fpar_mean).addBands(modis_lai_mean).addBands(dem_resampled).addBands(slope_resampled).addBands(aspect_resampled).addBands(modis_ndvi_mean_annual).addBands(modis_evi_mean_annual).addBands(modis_ndwi_mean_annual).addBands(modis_fpar_mean_annual).addBands(modis_lai_mean_annual).addBands(terra_gpp_mean_annual).addBands(aqua_gpp_mean_annual).addBands(terra_npp_mean_annual).addBands(aqua_npp_mean_annual)

    # 5. Apply masks based on slope and landcover

    # Create slope mask
    slope_mask = slope.lte(30).eq(1)
    slope_mask_reproj = slope_mask.reproject(crs=slope.projection())
    combined_image = combined_image.updateMask(slope_mask_reproj)

    lc_image = forest_in_co.select("landcover")
    lc_mask = lc_image.eq(41).Or(lc_image.eq(42)).Or(lc_image.eq(43))
    lc_points = lc_image.updateMask(lc_mask).sample(region=roi, projection=modis_projection, geometries=True)
    feature_spec = lc_points.getInfo()

    if len(feature_spec["features"]) == 0:
      print("Skipping for index {} since there are no values".format(index_to_run))
      return


    combined_image = combined_image.updateMask(lc_mask)
    modis_points = combined_image.sample(region=roi, projection=modis_projection, geometries=True)

    # Define functions to create bounding circles and boxes
    def bounding_circle_func(feature):
      intermediate_buffer = feature.buffer(231.5)
      return intermediate_buffer

    def bounding_box_func(feature):
      intermediate_box = feature.bounds()
      return intermediate_box

    bounding_circles = modis_points.map(bounding_circle_func)
    bounding_boxes = bounding_circles.map(bounding_box_func)
    sampledPointsCollectionWithAll = bounding_boxes.map(samplePointsFromFeatureWithAll, True)

    fileName = 'Forest_SD_{}_year_{}'.format(index_to_run, year_)
    description = 'Forest_{}_year_{}'.format(index_to_run, year_)
    export_task = ee.batch.Export.table.toDrive(
        collection=sampledPointsCollectionWithAll,
        description=description,
        folder=GDRIVE_FOLDER,
        fileNamePrefix=fileName,
        fileFormat='CSV'
      )
    export_task.start()
    while export_task.active():
        task_status = export_task.status()
        time.sleep(10)
    print("task completed for {}".format(index_to_run))
  except Exception as exp:
    print(exp)
    print("task failed for {} with {}".format(index_to_run, exp))

In [ ]:
STATE_ID = 5 # 5 == South Dakota
no_of_prc = 20

In [ ]:
region = make_regions_to_extract_modis(state_id=STATE_ID) 
number_of_patches = region.size().getInfo()
matrix = [[STATE_ID, index, region] for index in range(0, number_of_patches)]
if __name__ == '__main__':
  try:
    with Pool(no_of_prc) as p:
        p.map(run_mp, matrix)
  except Exception as exp:
    print(exp)
    print("Error in something")